In [ ]:
# control_plane_demo.py
#
# Minimal demo of the Enterprise AI Control Plane V1 inside Snowflake Notebooks.

from control_plane.config import ControlPlaneConfig
from control_plane.registry import ControlPlaneRegistry
from control_plane.agents import AtlasAgent, IntelligenceAgent
from control_plane.policies import Policy, PolicyEngine
from control_plane.workflows import WorkflowEngine
from control_plane.observability import RunLogger

# Get Snowpark session
session = get_active_session()

# --- 1. Config & logger -------------------------------------------------------

config = ControlPlaneConfig(
    environment="dev",
    run_log_table="CONTROL_PLANE_DB.CONTROL_PLANE_MONITORING.RUN_LOG",
)

logger = RunLogger(session=session, run_log_table=config.run_log_table)

# --- 2. Agents (stubs / integration points) -----------------------------------

# Stub intelligence agent impl: replace with actual import from your repo.
class DummyIntelligenceAgentImpl:
    def run(self, query: str):
        return {
            "summary": f"Dummy explanation for: {query}",
            "trace": []
        }

intel_impl = DummyIntelligenceAgentImpl()

intel_agent = IntelligenceAgent(
    name="snowflake_intel_agent_v2",
    description="Multi-modal reasoning agent on Snowflake.",
    session=session,
    agent_impl=intel_impl,
)

atlas_agent = AtlasAgent(
    name="atlas_mlops",
    description="Atlas MLOps control functions.",
    session=session,
)

# --- 3. Policy engine ---------------------------------------------------------

drift_policy = Policy(
    name="DRIFT_POLICY_FRAUD_MODEL",
    description="Trigger retrain when observed drift exceeds max_drift.",
    conditions={"max_drift": 0.15},
    actions=["trigger_retrain", "notify_owner"]
)

policy_engine = PolicyEngine(
    policies={"DRIFT_POLICY_FRAUD_MODEL": drift_policy}
)

# --- 4. Workflow engine & example workflow ------------------------------------

engine = WorkflowEngine(
    logger=logger,
    policy_engine=policy_engine,
)

def drift_management_workflow(context, policy_engine, logger, run_id):
    """
    Example Control Plane workflow:

    1. Ask Atlas to check drift (stub call here).
    2. Evaluate drift policy.
    3. If triggered:
       - call Atlas to retrain
       - ask Intelligence Agent for a human-readable explanation
    """

    # 1) Get drift value from context (in real life: CALL Atlas)
    observed_drift = context.get("observed_drift", 0.0)

    policy_result = policy_engine.evaluate(
        "DRIFT_POLICY_FRAUD_MODEL",
        context={"observed_drift": observed_drift},
    )

    steps = {
        "observed_drift": observed_drift,
        "policy_result": policy_result,
        "actions_taken": [],
    }

    # 2) Act if policy says so
    if policy_result["should_act"]:
        if "trigger_retrain" in policy_result["actions"]:
            atlas_result = atlas_agent.run(action="trigger_retrain")
            steps["actions_taken"].append(
                {"type": "atlas_retrain", "result": atlas_result}
            )

        # Call Intelligence Agent to summarize situation for humans
        intel_result = intel_agent.run(
            query=(
                f"Observed model drift of {observed_drift}. "
                f"Policy triggered retraining. "
                f"Explain this decision to an executive."
            )
        )
        steps["actions_taken"].append(
            {"type": "intel_summary", "result": intel_result}
        )

    return steps

engine.register_workflow(
    name="fraud_drift_management",
    description="End-to-end fraud model drift management workflow.",
    handler=drift_management_workflow,
)

# --- 5. Run the workflow -------------------------------------------------------

demo_context = {"observed_drift": 0.22}

result = engine.run_workflow(
    name="fraud_drift_management",
    context=demo_context,
)

print("=== CONTROL PLANE WORKFLOW RESULT ===")
print(result)